In [74]:
# Import libraries
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage.measurements import find_objects
from scipy import ndimage
from scipy.ndimage.filters import gaussian_filter
import xml.etree.ElementTree as ET
import csv

ship_bbs = [] #list of ship bounding boxes
train_list = []
val_list = []
last_fid = None #last file id
with open("train_ship_segmentations_v2.csv", "r") as f: #this file can be optained from the kaggle challenge
    reader = csv.reader(f, delimiter="\t") #process the CSV file
    for i, line in enumerate(reader):
        if(i % 1000 == 0): #print current status after every 1000 lines
            print(i)
        if(i == 0): #skip the first line (it contains header information)
            continue
        line = line[0].split(",") #split the image filename and the position of the ship (see documentation for the exact data format)
        if(len(line[1]) == 0): #skip this line if no ship on the image
            continue
        fid = line[0] #file id
        if last_fid != None and fid != last_fid: #check if we are processing a new image or another ship on the same image
            group = "val" if last_fid[0] == "e" or last_fid[0] == "f" else "train" #sort into train and validation folder based on the filename's first character
            annotation(filename=last_fid, ships=ship_bbs, size_of_image=[768,768], 
                       folder=group) #create the annotation
            if(group == "val"): #add to the apropriate list
                val_list.append(fid[:-4])
            else:
                train_list.append(fid[:-4])
            ship_bbs.clear() #get ready for the new image
        last_fid = fid
        rle = line[1].split(" ") #separate the ship location data
        rle = list(map(lambda w: int(w), rle)) #convert to int
        xmin = 999 #dummy default value
        xmax = -1
        ymin = 999
        ymax = -1
        for j in range(len(rle)//2): #contains pair of numbers 
            y1 = (rle[j*2]-1)%768 #tranform the data into (x, y) coordinates
            x = (rle[j*2]-1)//768
            y2 = rle[j*2+1]+y1-1
            if(y2 >= 768): #there is some error in the data if the coordinate is larger than 768 (the images are: 768x768)
                print("error:",i)
            ymin = min(ymin, y1)
            ymax = max(ymax, y2)
            xmin = min(xmin, x)
            xmax = max(xmax, x)
        ship_bbs.append([xmin,ymin,xmax,ymax]) #add the ship to the list
        
#repeat for the last image in the list    
group = "val" if last_fid[0] == "e" or last_fid[0] == "f" else "train"
annotation(filename=last_fid, ships=ship_bbs, size_of_image=[768,768], 
           folder=group)
if(group == "val"):
    val_list.append(fid[:-4])
else:
    train_list.append(fid[:-4])

#save the train list
with open("crop/train_list.txt", "w") as f:
    f.write("\n".join(train_list))

#save the validation list
with open("crop/val_list.txt", "w") as f:
    f.write("\n".join(val_list))

def annotation(size_of_image, ships, folder, filename): #Create the XML file with the annotation data
    #The XML file's format is identical with the ssd_keras sample annotation
    '''
    <annotation>
    <folder>train</folder>
    <filename>FILENAME.jpg</filename>
    <size>
        <width>768</width>
        <height>768</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>ship</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>XMIN</xmin>
            <ymin>YMIN</ymin>
            <xmax>XMAX</xmax>
            <ymax>YMAX</ymax>
        </bndbox>
    </object>
    </annotation>
    '''
    
    #create the XML file with the same structure
    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "folder").text = folder
    ET.SubElement(annotation, "filename").text = filename

    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(size_of_image[0])
    ET.SubElement(size, "height").text = str(size_of_image[1])
    ET.SubElement(size, "depth").text = str(3)

    ET.SubElement(annotation, "segmented").text = str(0)

    for ship in ships:
        object_ = ET.SubElement(annotation, "object")
        ET.SubElement(object_, "name").text = "ship"
        ET.SubElement(object_, "pose").text = "Unspecified"
        ET.SubElement(object_, "truncated").text = str(0)
        ET.SubElement(object_, "difficult").text = str(0)
        bndbox = ET.SubElement(object_, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(ship[0])
        ET.SubElement(bndbox, "ymin").text = str(ship[1])
        ET.SubElement(bndbox, "xmax").text = str(ship[2])
        ET.SubElement(bndbox, "ymax").text = str(ship[3])

    tree = ET.ElementTree(annotation)
    tree.write("crop/annotation/" + filename[0:-4] + ".xml") #save the XML


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
